## <center>对抗神经网络</center>

对抗神经网络其实是两个网络的组合, 可以理解为一个网络生成模拟数据, 另一个网络用于判断生成的数据是真实的还是模拟的. 生成模拟数据的网络要不断的优化自己让判别网络判断不出来, 判别网络也要优化自己让自己判断的更加准确. 所以二者形成了对抗关系, 因此也叫对抗神经网络.

实验证明, 利用这种网络间的对抗关系所形成的网络, 在无监督以及半监督领域取得了很好的效果, 可以算是用网络来监督网络的一个自学习过程. 

###  1 GAN的理论知识
GAN 由 generator (生成模型) 和 discriminator (判别模型) 两部分组成.
+ generator: 主要从训练数据集中产生相同分布的 sample, 对于输入 x, 类别标签 y, 在生成式模型中估计其联合概率分布 (两个及以上随机变量组成的随机向量的概率分布) .<br/><br/>
+ discriminator: 判断输入是真实数据还是 generator 生成的数据, 也就是估计样本属于某个类的条件概率, 它采用传统的监督学习方法

二者结合后, 经过大量的迭代训练, 让 generator 尽可能模拟出以假乱真的样本, 而 discriminator 会有更精确的鉴别真伪数据的能力, 最终整个 GAN 会达到所谓的纳什均衡, 即 discriminator 对于 generator 的数据鉴别结果正确率和错误率各占50%.

GAN 的网络结构如图所示
<center><img src='./img/17/1.png' width="500"/></center>

+ 生成模型也叫生成器. 它先用一个随机编码向量来输出一个模拟样本.<br/><br/>
+ 判别模型也叫判别器. 它的输入是一个样本 (可以是真实样本也可以是模拟样本), 输出一个判断该样本是真实样本还是模拟样本 (假样本) 的结果.

### 2 InfoGAN: 一种生成指定类别样本的GAN
GAN 的生成器在构建样本时使用了任意的噪音向量 $z$, 并从低维的噪音数据 $z$ 中还原出高维的样本数据. 这说明数据 $z$ 中包含了具有与样本相同的特征. 由于随意使用的噪音都能还原出高维样本数据, 表明噪声是由样本特征和非样本特征组成.

InfoGAN 是一种能够学习样本中的关键信息的 GAN. 具体的网络结构如下, 相比于对抗自编码网络, InfoGAN 的思路正好相反, InfoGAN 是先固定标准高斯分布作为网络输入, 再慢慢调整网络的输出去匹配复杂的样本分布.
<center><img src="./img/17/2.png" width="600"/></center>

InfoGAN 的理论思想是将输入的随机标准高斯分布当成噪音数据, 并将噪音分为两类, 第一类是不可压缩的噪音 $Z$, 第二类是可解释的信息 $C$. 

### 3 InfoGAN 生成 MNIST数据

#### 3.1 引入头文件

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow.contrib.slim as slim
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/")

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


#### 3.2 网络结构介绍
建立两个噪音数据 (一般噪音和隐含信息) 与 label 结合放到生成器中, 生成模拟样本, 然后将模拟样本和真实样本分布输入到判别器中, 生成判别结果、重构的隐含信息、样本标签.

在优化时, 让判别器对真实的样本判别结果为1, 对模拟数据的判别结果为0, 然后计算其中的损失,用于优化模型.

#### 3.3 定义生成器和判别器

In [2]:
def generator(x):
    with tf.variable_scope('generator',reuse=tf.AUTO_REUSE):
        x = slim.fully_connected(x,1024)
        x = slim.batch_norm(x,activation_fn=tf.nn.relu)
        x = slim.fully_connected(x,7*7*128)
        x = slim.batch_norm(x,activation_fn=tf.nn.relu)
        x = tf.reshape(tensor=x,shape=[-1,7,7,128])
        x = slim.conv2d_transpose(x,64,kernel_size=[4,4],stride=2,activation_fn=None)
        x = slim.batch_norm(x,activation_fn=tf.nn.relu)
        z = slim.conv2d_transpose(x,1,kernel_size=[4,4],stride=2,activation_fn=tf.nn.sigmoid)     
        return z

def leaky_relu(x):
    return tf.where(tf.greater(x,0),x,0.01*x)
# num_classes 表示标签维度
# num_cont 表示隐含信息维度
def discriminator(x,num_classes=10,num_info=2):
    with tf.variable_scope('discriminator',reuse=tf.AUTO_REUSE):
        x = tf.reshape(x,shape=[-1,28,28,1])
        x = slim.conv2d(x,num_outputs=64,kernel_size=[4,4],stride=2,activation_fn=leaky_relu)
        x = slim.conv2d(x,num_outputs=128,kernel_size=[4,4],stride=2,activation_fn=leaky_relu)
        x = slim.flatten(x)
        # 中间增加了一个隐含信息层,可以由真实样本也可以由模拟样本产生.
        shared_tensor = slim.fully_connected(x,num_outputs=1024,activation_fn=leaky_relu)
        # 与输入的x对比重构,计算损失
        recog_shared = slim.fully_connected(shared_tensor,num_outputs=128,activation_fn=leaky_relu)
        # 判别器用于计算预测结果
        discriminator = slim.fully_connected(shared_tensor,num_outputs=1,activation_fn=None)
        discriminator = tf.squeeze(discriminator,-1)
        # 判断重构类别
        recog_label = slim.fully_connected(recog_shared,num_outputs=num_classes,activation_fn=None)
        # 判断重构包含的info是否与输入的相同
        recog_info = slim.fully_connected(recog_shared,num_outputs=num_info,activation_fn=tf.nn.sigmoid)
        return discriminator,recog_label,recog_info
        

#### 3.4 定义网络模型
这里定义噪声的维度为38, 隐含信息维度为2, 二者都是符合标准高斯分布的随机数. 将他们与 one_hot 转换后的标签连接在一起放到生成器中.

In [3]:
batch_size = 10
classes_dim = 10
info_dim = 2
rand_dim = 38
n_input = 784

x = tf.placeholder(dtype=tf.float32,shape=[None,n_input])
y = tf.placeholder(dtype=tf.int32,shape=[None])

# 全部的输入数据
z_info = tf.random_normal(dtype=tf.float32,shape=[batch_size,info_dim])
z_rand = tf.random_normal(dtype=tf.float32,shape=[batch_size,rand_dim])
z = tf.concat(axis=1,values=[tf.one_hot(y,depth=classes_dim),z_info,z_rand])

# 生成一个模拟数据
gen = generator(z)
gen_out = tf.squeeze(gen,-1)

# 判别器的标准结果
y_real = tf.ones(batch_size) # 真
y_fake = tf.zeros(batch_size) # 假

discriminator_real,class_real,_ = discriminator(x)
discriminator_fake,class_fake,info_fake = discriminator(gen)
pred_class = tf.argmax(input=class_fake,axis=1)

#### 3.5 定义损失函数与优化器
判别器中, 判别结果的 loss 有两个: 真实输入的结果与模拟输入的结果. 将二者结合在一起生成 loss_d. 生成器的 loss 为自己输出的模拟数据, 让它在判别器中为真, 定义为 loss_g.

然后还要定义网络中共有的 loss 值: 真实标签与输入真实样本判别出的标签, 真实标签与输入模拟样本判别出的标签, 隐含信息的重构误差. 然后创建两个优化器, 将它们放到对应的优化器中.

在这里, 需要将判别器的学习率设小, 将生成器的学习率设大一些. 由于生成器需要在随机数据中寻找合适的值, 而判别器训练的数据都是真实样本, 这种做法可以让生成器更快的进化速度来模拟真实数据.

In [4]:
#　判别器 loss
loss_d_r = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=discriminator_real,labels=y_real))
loss_d_f = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=discriminator_fake,labels=y_fake))
loss_d = (loss_d_r+loss_d_f)/2
# 生成器 loss
loss_g = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=discriminator_fake,labels=y_real))

# 计算 factor loss
# 分类正确,但是生成的样本错误
loss_cf = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=class_fake,labels=y))
# 生成的样本与分类都正确, 但是与输入的分类对不上
loss_cr = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=class_real,labels=y))
loss_c = (loss_cf+loss_cr)/2
# 隐含变量的 loss
loss_info = tf.reduce_mean(tf.square(info_fake-z_info))

train_discriminator = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_d+loss_c+loss_info)
train_generator = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_g+loss_c+loss_info)

#### 3.6 开始训练与测试
建立 session, 在循环里面使用 run 来运行前面定义的优化器

In [5]:
training_epochs = 3
display_step = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feeds = {x:batch_xs,y:batch_ys}
            l_disc,_ = sess.run([loss_d,train_discriminator],feed_dict=feeds)
            l_gen,_  = sess.run([loss_g,train_generator],feed_dict=feeds)
        # 开始训练
        if epoch % display_step == 0:
            print("epoch: ", '%04d' % (epoch+1),'cost=',"{:.9f}".format(l_disc),l_gen)
    print("result:",loss_d.eval({x:mnist.test.images[:batch_size],y:mnist.test.labels[:batch_size]}),
         loss_g.eval({x:mnist.test.images[:batch_size],y:mnist.test.labels[:batch_size]}))

epoch:  0001 cost= 5.372066021 0.00027211825
epoch:  0002 cost= 18.071308136 1.0149408e-10
epoch:  0003 cost= 49.293067932 5.249316e-28
result: 44.583153 4.9828908e-24
